In [63]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [64]:
#Criando uma session com acesso ao gcp
spark = (
    SparkSession
    .builder
    .appName("Desafio-edc-format-trusted")
    .config("spark.serializer","org.apache.spark.serializer.KryoSerializer")
    .getOrCreate()
)

In [65]:
""" .config("spark.jars","./jars/gcs-connector-hadoop3-latest.jar")
    .config("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
    .config("fs.AbstractFileSystem.gs.impl","com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
    .config("fs.gs.auth.service.account.enable","true")
    .config("fs.gs.auth.service.account.json.keyfile", "/mnt/d/EngDados/gcp/gcp-estudos-engdados-20f2cdfffed8.json")
    .config("spark.driver.memory", "8g")
"""

' .config("spark.jars","./jars/gcs-connector-hadoop3-latest.jar")\n    .config("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")\n    .config("fs.AbstractFileSystem.gs.impl","com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")\n    .config("fs.gs.auth.service.account.enable","true")\n    .config("fs.gs.auth.service.account.json.keyfile", "/mnt/d/EngDados/gcp/gcp-estudos-engdados-20f2cdfffed8.json")\n    .config("spark.driver.memory", "8g")\n'

## Formatação dos dados de CNAE - Atividade Economica
Ajustar os tipos de dados para melhor leitura/joins

In [66]:
df_cnae = spark.read.parquet("gs://bootcamp-edc/raw/cnae.parquet/*.parquet")

In [67]:
df_cnae.limit(5).toPandas()

,cod_atividade,desc_atividade
0,0111301,Cultivo de arroz
1,0111302,Cultivo de milho
2,0111303,Cultivo de trigo
3,0111399,Cultivo de outros cereais não especificados an...
4,0112101,Cultivo de algodão herbáceo


In [68]:
df_cnae = (
    df_cnae
    .withColumn("cod_atividade", f.col("cod_atividade").cast("int"))
    .withColumn("desc_atividade", f.trim(f.col("desc_atividade")))
    )

In [69]:
df_cnae.limit(10).toPandas()

,cod_atividade,desc_atividade
0,111301,Cultivo de arroz
1,111302,Cultivo de milho
2,111303,Cultivo de trigo
3,111399,Cultivo de outros cereais não especificados an...
4,112101,Cultivo de algodão herbáceo
5,112102,Cultivo de juta
6,112199,Cultivo de outras fibras de lavoura temporária...
7,113000,Cultivo de cana-de-açúcar
8,114800,Cultivo de fumo
9,115600,Cultivo de soja


In [70]:
df_cnae.printSchema()

root
 |-- cod_atividade: integer (nullable = true)
 |-- desc_atividade: string (nullable = true)



In [71]:
df_cnae.write.mode("overwrite").parquet("gs://bootcamp-edc/trusted/cnae.parquet")

In [72]:
del(df_cnae)

## Leitura dos dados de Município

In [91]:
df_municipio = spark.read.parquet("gs://bootcamp-edc/raw/municipio.parquet/*.parquet")

In [74]:
df_municipio.limit(5).toPandas()

,cod_municipio,desc_municipio
0,0001,GUAJARA-MIRIM
1,0002,ALTO ALEGRE DOS PARECIS
2,0003,PORTO VELHO
3,0004,BURITIS
4,0005,JI-PARANA


In [75]:
df_municipio = (
    df_municipio
    .withColumn("cod_municipio", f.col("cod_municipio").cast("int"))
    .withColumn("desc_municipio", f.initcap(f.trim(f.col("desc_municipio"))))
)

In [76]:
df_municipio.limit(10).toPandas()

,cod_municipio,desc_municipio
0,1,Guajara-mirim
1,2,Alto Alegre Dos Parecis
2,3,Porto Velho
3,4,Buritis
4,5,Ji-parana
5,6,Chupinguaia
6,7,Ariquemes
7,8,Cujubim
8,9,Cacoal
9,10,Nova Uniao


In [77]:
df_municipio.printSchema()

root
 |-- cod_municipio: integer (nullable = true)
 |-- desc_municipio: string (nullable = true)



In [78]:
df_municipio.write.mode("overwrite").parquet("gs://bootcamp-edc/trusted/municipio.parquet")

In [79]:
del(df_municipio)

## Leitura dos dados de estabelecimentos

In [98]:
df_estabelecimento = spark.read.parquet("gs://bootcamp-edc/raw/estabelecimento.parquet/*.parquet")

In [81]:
df_estabelecimento.limit(5).toPandas()

,cnpj_basico,cnpj_ordem,cnpj_dig_verificador,idc_matriz_filial,nome_fantasia,cod_situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_cidade_exterior,cod_pais,...,cod_municipio,ddd_telefone_1,num_telefone_1,ddd_telefone_2,num_telefone_2,ddd_fax,num_fax_2,email,situacao_especial,data_situacao_especial
0,12161981,0001,38,1,CASA DA PIZZA PAULISTA,02,20100705,00,None,None,...,3613,73,99981255,None,None,None,None,CASADAPIZZAPAULISTA@HOTMAIL.COM,None,None
1,15674191,0001,44,1,UNIVERSO CONSTRUCOES,08,20161007,01,None,None,...,9701,61,30324570,None,None,None,None,None,None,None
2,08768885,0001,85,1,MERCADO COMBOIO,08,20100324,01,None,None,...,9701,61,33337177,None,None,61,33337177,None,None,None
3,12162323,0001,60,1,FIGUEIRA SELF SERVICE,04,20190116,63,None,None,...,2549,81,37311762,None,None,None,None,None,None,None
4,13415367,0001,18,1,None,02,20110301,00,None,None,...,8883,54,32911464,None,None,None,None,None,None,None


In [82]:
df_estabelecimento.filter(df_estabelecimento.data_inicio_atividade == "4399103").limit(5).toPandas()

,cnpj_basico,cnpj_ordem,cnpj_dig_verificador,idc_matriz_filial,nome_fantasia,cod_situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_cidade_exterior,cod_pais,...,cod_municipio,ddd_telefone_1,num_telefone_1,ddd_telefone_2,num_telefone_2,ddd_fax,num_fax_2,email,situacao_especial,data_situacao_especial


In [83]:
df_estabelecimento.filter(df_estabelecimento.data_situacao_especial.isNotNull()).limit(5).toPandas()

KeyboardInterrupt: 

In [99]:
#Formata colunas inteiras
int_columns = ["idc_matriz_filial","cod_situacao_cadastral","motivo_situacao_cadastral","cod_pais","cod_municipio","cnae_fiscal_principal","cnae_fiscal_secundaria"]
for column in int_columns:
    df_estabelecimento = df_estabelecimento.withColumn(column, f.col(column).cast("int"))

In [100]:
#Formata colunas strings
str_columns = ["nome_fantasia","nome_cidade_exterior","nome_logradouro","complemento_logradouro","complemento_logradouro","nome_bairro"]
for column in str_columns:
    df_estabelecimento = df_estabelecimento.withColumn(column, f.initcap(f.trim(f.col(column))))
    
df_estabelecimento = df_estabelecimento.withColumn("email", f.lower(f.trim(f.col("email"))))

In [95]:
"""spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
dat_columns = ["data_situacao_cadastral","data_inicio_atividade","data_situacao_especial"]
for column in dat_columns:
    df_estabelecimento = df_estabelecimento.withColumn(column,f.to_date(f.col(column),"yyyyMMdd"))
"""

In [88]:
df_estabelecimento.limit(10).toPandas()

,cnpj_basico,cnpj_ordem,cnpj_dig_verificador,idc_matriz_filial,nome_fantasia,cod_situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_cidade_exterior,cod_pais,...,cod_municipio,ddd_telefone_1,num_telefone_1,ddd_telefone_2,num_telefone_2,ddd_fax,num_fax_2,email,situacao_especial,data_situacao_especial
0,12161981,0001,38,1,Casa Da Pizza Paulista,2,2010-07-05,0,None,NaN,...,3613,73,99981255,None,None,None,None,casadapizzapaulista@hotmail.com,None,None
1,15674191,0001,44,1,Universo Construcoes,8,2016-10-07,1,None,NaN,...,9701,61,30324570,None,None,None,None,None,None,None
2,08768885,0001,85,1,Mercado Comboio,8,2010-03-24,1,None,NaN,...,9701,61,33337177,None,None,61,33337177,None,None,None
3,12162323,0001,60,1,Figueira Self Service,4,2019-01-16,63,None,NaN,...,2549,81,37311762,None,None,None,None,None,None,None
4,13415367,0001,18,1,None,2,2011-03-01,0,None,NaN,...,8883,54,32911464,None,None,None,None,None,None,None
5,10282156,0001,66,1,Marta Representacoes,2,2021-06-09,0,None,NaN,...,1761,84,32058887,None,None,None,None,sertecnatal@uol.com.br,None,None
6,13415924,0001,09,1,None,4,2019-02-27,63,None,NaN,...,6291,19,32269485,None,None,None,None,luewesleysilva@yahoo.com.br,None,None
7,16764264,0001,51,1,Seven Distribuicoes E Logistica,8,2017-01-05,1,None,NaN,...,4305,32,88623163,None,None,None,None,sevendistribuicoes@gmail.com,None,None
8,10970575,0001,90,1,M.r. Braga,2,2009-07-09,0,None,NaN,...,6001,21,23015583,21,23015581,None,None,spjauditoria@uol.com.br,None,None
9,15675603,0001,60,1,Lu Moda,4,2021-03-03,63,None,NaN,...,841,98,87011813,None,None,None,None,None,None,None


In [96]:
df_estabelecimento.printSchema()

root
 |-- cnpj_basico: string (nullable = true)
 |-- cnpj_ordem: string (nullable = true)
 |-- cnpj_dig_verificador: string (nullable = true)
 |-- idc_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- cod_situacao_cadastral: integer (nullable = true)
 |-- data_situacao_cadastral: date (nullable = true)
 |-- motivo_situacao_cadastral: integer (nullable = true)
 |-- nome_cidade_exterior: string (nullable = true)
 |-- cod_pais: integer (nullable = true)
 |-- data_inicio_atividade: date (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: integer (nullable = true)
 |-- tipo_logradouro: string (nullable = true)
 |-- nome_logradouro: string (nullable = true)
 |-- numero_logradouro: string (nullable = true)
 |-- complemento_logradouro: string (nullable = true)
 |-- nome_bairro: string (nullable = true)
 |-- cod_cep: string (nullable = true)
 |-- sigla_uf: string (nullable = true)
 |-- cod_municipio: integ

In [ ]:
df_estabelecimento.createOrReplaceTempView("tab_estabelecimento")

In [ ]:
spark.sql("""
select max(data_situacao_cadastral) as data_situacao_cadastral_max
, min(data_situacao_cadastral) as data_situacao_cadastral_min
, max(data_situacao_especial) as data_situacao_especial_max
, min(data_situacao_especial) as data_situacao_especial_min
from tab_estabelecimento
""").show()

In [101]:
df_estabelecimento.write.mode("overwrite").parquet("gs://bootcamp-edc/trusted/estabelecimento.parquet")

In [ ]:
del(df_estabelecimento)

In [ ]:
spark.stop()